# A multi-LLM pipeline. 

In this lab, I will evaluate DeepSeek answers using Gemini. 

In [1]:
# you need to run the lab2 before this

%store -r name
%store -r summary
%store -r resume_md
%store -r system_prompt

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

/Users/shubhamlad/Desktop/Shubham/AgentPractice/Courses/agent-practice/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv(override=True)

True

In [4]:
# create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [5]:

evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and Resume details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## Resume:\n{resume_md}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [6]:
evaluator_system_prompt

"You are an evaluator that decides whether a response to a question is acceptable. You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. The Agent is playing the role of Shubham Lad and is representing Shubham Lad on their website. The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. The Agent has been provided with context on Shubham Lad in the form of their summary and Resume details. Here's the information:\n\n## Summary:\nWith extensive experience in both product and service-based companies as a skilled Software Engineer, I thrive in fast-paced environments where results matter. My expertise in startup environments has given me a deep understanding of what it takes to deliver innovative solutions that meet client needs.\n\nI have a proven track record of optimizing complex backend systems and deli

In [7]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [8]:
gemini_client = OpenAI(
    api_key=os.environ.get("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai"
)

In [9]:
# Here we will use the parse method so that we will get the responses in the predefined structure

def evaluate(reply, message, history):
    user_prompt = evaluator_user_prompt(reply, message, history)
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": user_prompt}]
    response = gemini_client.beta.chat.completions.parse(
        model="gemini-2.0-flash",
        messages=messages,
        response_format=Evaluation
    )
    evaluation = response.choices[0].message.parsed
    return evaluation

In [10]:
deepseek_client = OpenAI(
    api_key=os.environ.get("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)
model_name = "deepseek/deepseek-chat-v3.1:free"

In [27]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold any patent?"}]
response = deepseek_client.chat.completions.create(
    model=model_name,
    messages=messages,
)
reply = response.choices[0].message.content

In [28]:
reply

"No, I don't hold any patents. My work has primarily focused on delivering high-quality software solutions and optimizing systems for performance and scalability, rather than pursuing patents. However, I have successfully led and contributed to several projects that required innovative solutions, particularly in the areas of data processing, cloud infrastructure, and full-stack development.\n\nIf you're interested in a specific area where you think a patent might be relevant, I'd be happy to discuss the technical challenges and solutions we implemented!"

In [29]:
evaluate(reply, messages[-1]["content"], messages[:1])

Evaluation(is_acceptable=True, feedback="The response is great. It accurately answers the question and then provides additional context as to why Shubham doesn't have any patents, while still highlighting his innovative skills and contributions to projects. It is also professional and engaging as requested.")

In [11]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the qualitycontrol rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = deepseek_client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    new_reply = response.choices[0].message.content
    return new_reply

In [12]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\nEverything in your reply needs to be in pig latin -\
            it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt

    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = deepseek_client.chat.completions.create(model=model_name, messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        print(reply)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The agent's response is not acceptable. It seems to be gibberish and doesn't answer the user's question about patents. A professional response should directly address the query or politely state that the information is not available.
Otay, Iway avehay onay atentpay.
